# Algorytmika i matematyka uczenia maszynowego 
## Laboratorium 11

### Zadanie 1

Zaimplementuj systemu rekomendacji filmów w oparciu o indeks Jaccarda. System ma za zadanie zwrócić listę filmów sugerowany dla podanego użytownika.

Dane zostały pobrane z serwisu Kaggle z https://www.kaggle.com/datasets/gargmanas/movierecommenderdataset

Zbiór zawiera dwa pliki:
- `movies.csv` lista filmów wraz z ich identyfikatorami
- `ratings.csv` lista ocen filmów przez użytkowników

**Zadanie:**
* Wczytaj oba pliki.
* Zamień wszystkie oceny użytkownika na wartość 1 (zastosuj próg okreśjący czy film się podobał czy nie np. 3).
* Stwórz macierz ocen użytkowników w której wierszach będą użytkownicy, a w kolumnach filmy. Wartość w macierzy jest flagą mówiącą czy użytkownikowi film się podobał czy nie. 
* Wypełnij brakujące wartości zerami.
* Utwórz macierz podobieństwa Jaccarda pomiędzy użytkownikami (każdy z każdym).
    - Możesz wykorzystać funkcję [jaccard](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jaccard.html) z biblioteki scipy.
* Zaimplementuj funkcję która dla podanego użytkownika zwróci listę sugerowanych filmów.
    - Funkcja powinna zwrócić listę filmów które nie były ocenione przez użytkownika, a które są rekomendowane dla niego.



In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform, jaccard

#Wczytanie danych
movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")


#Zamiana na binarne - podoba sie, nie podoba sie
minimum = min(ratings_df['rating'])
maximum = max(ratings_df['rating'])

print(f"Minimalny rating: {minimum}")
print(f"Maksymalny rating: {minimum}")

threshold = 3

print(f"Threshold: {threshold}")

ratings_df['rating'] = (ratings_df['rating'] >= threshold).astype(int)

########
#Macierz

users = np.unique(ratings_df['userId']).tolist()
movies = list(movies_df['movieId'])

#Macierz uzytkownikow (wiersze) i filmow (kolumny)
user_movie_matrix = ratings_df.pivot(index='userId', columns='movieId', values='rating')

#zamien nan na 0
user_movie_matrix = user_movie_matrix.fillna(0)
#Do array
user_movie_matrix = np.array(user_movie_matrix)


#Odleglosc jaccarda przeciwne do podobienstwa
jaccard_distances = pdist(user_movie_matrix, metric='jaccard')

#zamiana odleglosci na podobienstwo - przy okazji transformacja do macierzy
jaccard_similarity = 1 - squareform(jaccard_distances)

print("-----------------------------------------------")
print("Jaccard matrix test:")
print("-----------------------------------------------")
#Test
is_symmetric = np.allclose(jaccard_similarity, jaccard_similarity.T)
print(f"Is symetrical: {is_symmetric}")

#Diagonala wypelniona -1 aby nie uwzgledniac porownania uzytkownika samego ze sobą
np.fill_diagonal(jaccard_similarity, -1)

max_sim_idx = np.unravel_index(np.argmax(jaccard_similarity), jaccard_similarity.shape)

print(f"User: {max_sim_idx[0]} and User: {max_sim_idx[1]} --> Similarity: {jaccard_similarity[max_sim_idx[0],max_sim_idx[1]]}")

user_0_movies = user_movie_matrix[max_sim_idx[0]]
user_1_movies = user_movie_matrix[max_sim_idx[1]]

user_0_mask = set([i for i, val in enumerate(user_0_movies) if val == 1])
user_1_mask = set([i for i, val in enumerate(user_1_movies) if val == 1])

common_idx = list(user_0_mask & user_1_mask)

common_movies = movies_df['title'][common_idx]
print("Common movies: ")
for c_movie in common_movies[0:5]:
    print(c_movie)


print("-----------------------------------------------")
print("Function to recommend movies: ")
print("-----------------------------------------------")
#Funkcja:
# - zwraca 5 filmow (aby nie zwracac zbyt wiele)
# - zwraca na podstawie podobienstwa uzytkownikow i filmow ktore oni ogladneli

def reccomend_movies_to_user(ratings_df, movies_df, user):
    users = np.unique(ratings_df['userId']).tolist()
    
    if user not in users:
        raise ValueError("User does not exist")
        
    users = [x for x in users if x != user]
    
    #Macierz uzytkownikow (wiersze) i filmow (kolumny)
    user_movie_matrix = ratings_df.pivot(index='userId', columns='movieId', values='rating')
    
    #zamien nan na 0
    user_movie_matrix = user_movie_matrix.fillna(0)
    #Do array
    user_movie_matrix = np.array(user_movie_matrix)
    
    dst_list = []
    for u_list in users:
        dst = jaccard(user_movie_matrix[user-1], user_movie_matrix[u_list-1])
        dst_list.append(1 - dst)
      
    #Sortowanie indeksami -  pierwsze wartosci indeksow to uzytkownicy o najbardziej podobnym guscie filmowym
    indexes = list(range(len(dst_list)))
    indexes.sort(key=lambda i: dst_list[i], reverse=True)
    
    closest_users = indexes[0:5]
    
    proposed_movies = []
    for u in closest_users:
        
        proposed_movies.append(user_movie_matrix[u])
        
    
    
    result = np.sum(proposed_movies, axis=0)
    indexes = list(range(len(dst_list)))
    indexes.sort(key=lambda i: result[i], reverse=True) 
    
    idx_5 = indexes[0:5]
    idx_5 = [x + 1 for x in idx_5]
    
    movies_5 = movies_df['title'][idx_5]
    
    return list(movies_5)

user = 129
movies_5 = reccomend_movies_to_user(ratings_df = ratings_df, movies_df = movies_df, user = user)

print(f"5 movies recommended for user: {user} are: {movies_5}")





Minimalny rating: 0.5
Maksymalny rating: 0.5
Threshold: 3
-----------------------------------------------
Jaccard matrix test:
-----------------------------------------------
Is symetrical: True
User: 129 and User: 467 --> Similarity: 0.625
Common movies: 
Toy Story (1995)
Pulp Fiction (1994)
Quiz Show (1994)
Die Hard: With a Vengeance (1995)
Santa Clause, The (1994)
-----------------------------------------------
Function to recommend movies: 
-----------------------------------------------
5 movies recommended for user: 129 are: ['Snow White and the Seven Dwarfs (1937)', 'Pocahontas (1995)', 'Mighty Aphrodite (1995)', 'Pushing Hands (Tui shou) (1992)', 'Shallow Grave (1994)']




### Zadanie 2


Algorytm MinHash na przykładzie wykrywania plagiatów

Wykonaj kolejno następujące kroki:

1. Pobierz 8 akapitów tekstu (nie za krótkich), każdy o różnej tematyce (mogą być np. z różnych haseł Wikipedii), trzymaj się jednego języka (np. PL lub ENG). Wklej je do jednego pliku tekstowego, z linią wolną jako separatorem.

2. Skopiuj wybrane 2-3 akapity i ręcznie nieco zmodyfikuj.

> Przykład (z hasła https://pl.wikipedia.org/wiki/Fryderyk_Chopin):

```Jest uważany za jednego z najwybitniejszych kompozytorów romantycznych, a także za jednego z najważniejszych polskich kompozytorów w historii. Był jednym z najsłynniejszych pianistów swoich czasów, często nazywany poetą fortepianu. Elementem charakterystycznym dla utworów Chopina jest pogłębiona ekspresja oraz czerpanie z wzorców stylistycznych polskiej muzyki ludowej.```

↓↓↓ Zmieniono na ↓↓↓

```Jest uważany za jednego z najwybitniejszych kompozytorów romantycznych, a także za jednego z najważniejszych kompozytorów polskich w historii.  Był jednym  z najsłynniejszych pianistów swoich czasów, często nazywany poetą fortepianu! Elementem charakterystycznym dla utworów Fryderyka Chopina jest pogłębiona ekspresja oraz czerpanie z wzorców polskiej muzyki ludowej.```

Otrzymasz zatem w pliku tekstowym 10 lub 11 akapitów tekstu (kolejność dowolna, te „splagiatowane” nie muszą być na końcu).

3. Z poziomu skryptu: wczytaj wszystkie akapity z pliku. Zbuduj 100 "losowych" funkcji haszujących.

> Sugestia: funkcją "bazową" jest po prostu `hash(...)`. Zakładamy 64-bitową wersję Pythona 3.x, wtedy `hash(...)` jest 64-bitowy.

Na liście seeds umieszczamy 100 losowych liczb 64-bitowych. Aby obliczyć $i$-ty hash dla ciągu 
należy wykonać `hash(s) ^ seeds[i]` (użycie operatora XOR).

4. Przyjmij niewielką wartość $Q$ (np. 15) i dla każdego akapitu
    - oznacz jego długość przez $n$,
    - dla każdej ze 100 funkcji haszujących policz hasza w przesuwnym oknie tekstu o długości $Q$ znaków (czyli łącznie mamy $n - Q + 1$ wartości hasza); zapamiętaj MINIMUM z tych $n - Q + 1$
 wartości.
Na wyjściu mamy zatem (dla 11 akapitów) 11 * 100 wartości haszy.

5. Rozważ pary akapitów "każdy z każdym". Jeśli dla danej pary co najmniej (np.) 30 haszy jest wspólnych, to uważamy akapity za podobne (być może plagiat) i wyświetlamy na ekranie.

6. Wyświetl czas obliczeń (powinien wynosić mniej niż 0.5s).

7. Poeksperymentuj z liczbą użytych funkcji haszujących, wartością, stopniem modyfikacji oryginalnych akapitów tekstu, progiem detekcji akapitów podobnych.

In [2]:
import random
import itertools
import time


with open('tekst.txt', 'r') as file:
    content = file.read()
    
    
paragraphs = [paragraph.strip() for paragraph in content.split('\n\n') if paragraph.strip()]

def MinHash(paragraphs, seeds_number, plagiary_treshold, window_size):
    
    seeds = [random.getrandbits(64) for _ in range(seeds_number)]
    q = window_size
    
    #Start time
    start_time = time.time()

    par_hash_list = []
    for par in paragraphs:
        #Petla dla kazdego akapitu
        n = len(par)

        
        final_hash_list = []
        #Petla dla kazdego seeda
        for seed in seeds:
            i = 0
            hash_list = []
            #Okno przesuwne
            while i <n-15+1:
                okno = par[i:i+q]
                
                h = hash(okno) ^ seed
                #Okno przesuwne, na kazde okno jeden hash
                hash_list.append(h)
                i+=1
                
            #Min hash z tych utworzonych z okna przesuwnego    
            min_hash = min(hash_list)
            
            #Lista 100 hashy dla akapitu
            final_hash_list.append(min_hash)
            
        #Lista 100 hashy dla kazdego akapitu  
        par_hash_list.append(final_hash_list)
            
        
    
    
    n_parahraphs = len(par_hash_list)
    combinations = list(itertools.combinations(range(n_parahraphs), 2))
    for i, j in combinations:
        
        common = len(set(par_hash_list [i]) & set(par_hash_list [j]))
        
        if common >= plagiary_treshold:
            print(f"Akapity {paragraphs[i][0:13]}... i {paragraphs[j][0:13]}... są podobne (wspólnych hashy: {common})")    
        
        
    t = round(time.time() - start_time,4)
    print(f"Czas wykonania: {t} s") 
        
        
print("Parameters: seed_number = 100 ; treshold = 30 ; window_size = 15 ")    
MinHash(paragraphs, seeds_number = 100, plagiary_treshold = 30, window_size = 15)    

print("\n")

print("Parameters: seed_number = 10 ; treshold = 30 ; window_size = 15 ")    
MinHash(paragraphs, seeds_number = 10, plagiary_treshold = 30, window_size = 15)    

print("\n")

print("Parameters: seed_number = 50 ; treshold = 5 ; window_size = 15 ")    
MinHash(paragraphs, seeds_number = 50, plagiary_treshold = 5, window_size = 15)   

print("\n")

print("Parameters: seed_number = 100 ; treshold = 5 ; window_size = 5 ")    
MinHash(paragraphs, seeds_number = 100, plagiary_treshold = 5, window_size = 5)     


Parameters: seed_number = 100 ; treshold = 30 ; window_size = 15 
Akapity Goryl[3] (Gor... i Goryl[3] (Gor... są podobne (wspólnych hashy: 53)
Akapity Surykatka sza... i Surykatka sza... są podobne (wspólnych hashy: 53)
Akapity Lew afrykaĹ„s... i Lew @frykaĹ„s... są podobne (wspólnych hashy: 43)
Czas wykonania: 0.1646 s


Parameters: seed_number = 10 ; treshold = 30 ; window_size = 15 
Czas wykonania: 0.0172 s


Parameters: seed_number = 50 ; treshold = 5 ; window_size = 15 
Akapity Szympans[17] ... i Goryl[3] (Gor... są podobne (wspólnych hashy: 5)
Akapity Szympans[17] ... i Goryl[3] (Gor... są podobne (wspólnych hashy: 5)
Akapity Goryl[3] (Gor... i Goryl[3] (Gor... są podobne (wspólnych hashy: 30)
Akapity Surykatka sza... i Surykatka sza... są podobne (wspólnych hashy: 36)
Akapity Lew afrykaĹ„s... i Lew @frykaĹ„s... są podobne (wspólnych hashy: 16)
Czas wykonania: 0.0848 s


Parameters: seed_number = 100 ; treshold = 5 ; window_size = 5 
Akapity Szympans[17] ... i Goryl[3] (Gor... są